In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('temps.csv')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


In [4]:
df.shape, df.describe

((348, 12),
 <bound method NDFrame.describe of      year  month  day   week  temp_2  temp_1  average  actual  forecast_noaa  \
 0    2016      1    1    Fri      45      45     45.6      45             43   
 1    2016      1    2    Sat      44      45     45.7      44             41   
 2    2016      1    3    Sun      45      44     45.8      41             43   
 3    2016      1    4    Mon      44      41     45.9      40             44   
 4    2016      1    5   Tues      41      40     46.0      44             46   
 ..    ...    ...  ...    ...     ...     ...      ...     ...            ...   
 343  2016     12   27   Tues      42      42     45.2      47             41   
 344  2016     12   28    Wed      42      47     45.3      48             41   
 345  2016     12   29  Thurs      47      48     45.3      48             43   
 346  2016     12   30    Fri      48      48     45.4      57             44   
 347  2016     12   31    Sat      48      57     45.5      40 

In [5]:
# One-hot encode the data using pandas get_dummies
df = pd.get_dummies(df)

# Display the first 5 rows of the last 12 columns
df.iloc[:,5:].head(5)

,average,actual,forecast_noaa,forecast_acc,forecast_under,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,45.6,45,43,50,44,29,1,0,0,0,0,0,0
1,45.7,44,41,50,44,61,0,0,1,0,0,0,0
2,45.8,41,43,46,47,56,0,0,0,1,0,0,0
3,45.9,40,44,48,46,53,0,1,0,0,0,0,0
4,46.0,44,46,46,46,41,0,0,0,0,0,1,0


In [6]:
import numpy as np

labels = np.array(df['actual'])

features= df.drop('actual', axis = 1)
feature_list = list(df.columns)
features = np.array(df)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [9]:
print('Training Features Shape:', x_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', x_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (261, 18)
Training Labels Shape: (261,)
Testing Features Shape: (87, 18)
Testing Labels Shape: (87,)


In [10]:
baseline_preds = x_test[:, feature_list.index('average')]
baseline_errors = abs(baseline_preds - y_test)
np.mean(baseline_errors)

5.05977011494253

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [14]:
predictions = rf.predict(x_test)

In [21]:
predictions.shape

(87,)

In [16]:
rf.score(x_test, y_test)

0.9984914706720045

In [23]:
from sklearn.tree import export_graphviz
import pydot
tree = rf.estimators_[5]
from sklearn.tree import export_graphviz
import pydot
tree = rf.estimators_[5]
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')